In [1]:
import numpy as np
import os

import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras import Sequential, Input
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
vis = pd.read_csv('data/air_visit_data.csv')
ares = pd.read_csv('data/air_reserve.csv')
hres = pd.read_csv('data/hpg_reserve.csv')
astore = pd.read_csv('data/air_store_info.csv')
hstore = pd.read_csv('data/hpg_store_info.csv')
sid = pd.read_csv('data/store_id_relation.csv')
dinfo = pd.read_csv('data/date_info.csv')

In [3]:
ares = pd.merge(ares, sid, how = 'left', on = ['air_store_id'])
hres = pd.merge(hres, sid, how = 'left', on = ['hpg_store_id'])
res = ares.append(hres)

res['visit_date'] = pd.to_datetime(res['visit_datetime']).dt.date

res = res.groupby(['air_store_id', 'visit_date']).sum().reset_index()

vis['visit_date'] = pd.to_datetime(vis['visit_date']).dt.date
dinfo['visit_date'] = pd.to_datetime(dinfo['calendar_date']).dt.date

data = pd.merge(res, vis, how = 'right', on = ['air_store_id', 'visit_date'])
data = pd.merge(data, dinfo, how = 'left', on = ['visit_date'])
data = pd.merge(data, astore, how = 'left', on = ['air_store_id'])

# months = {
#     1 : 'Jan',
#     2 : 'Feb',
#     3 : 'Mar',
#     4 : 'Apr',
#     5 : 'May',
#     6 : 'Jun',
#     7 : 'Jul',
#     8 : 'Aug',
#     9 : 'Sep',
#     10 : 'Oct',
#     11 : 'Nov',
#     12 : 'Dec',
# }

data['year'] = pd.to_datetime(data['visit_date']).dt.year
# data['month'] = pd.to_datetime(data['visit_date']).dt.month.map(months)

data = pd.concat([data ,pd.get_dummies(data['air_genre_name'])], axis = 1)
data = pd.concat([data ,pd.get_dummies(data['air_area_name'])], axis = 1)
data = pd.concat([data ,pd.get_dummies(data['day_of_week'])], axis = 1)
# data = pd.concat([data ,pd.get_dummies(data['month'])], axis = 1)

data = data.fillna(0)
data = data.drop(['air_store_id', 'latitude', 'longitude', 'visit_date', 'calendar_date', 'air_genre_name', 'air_area_name', 'day_of_week'], axis = 1)

data['visitors'] = np.log1p(data['visitors'])

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
train_X = data[data['year'] == 2016].drop(['visitors', 'year'], axis = 1)
train_Y = data[data['year'] == 2016]['visitors']
test_X = data[data['year'] == 2017].drop(['visitors', 'year'], axis = 1)
test_Y = data[data['year'] == 2017]['visitors']

In [5]:
model = Sequential()

model.add(Dense(128, activation = 'relu', input_shape = (train_X.shape[1],)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.compile(loss = 'mse', optimizer = Adam(lr = 0.001, decay = 0.0001), metrics = ['mean_squared_logarithmic_error'])
model.fit(x = train_X, y = train_Y, epochs = 50, batch_size = 512)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
174535/174535 [==============================] - 4s 23us/step - loss: 0.9512 - mean_squared_logarithmic_error: 0.1116
Epoch 2/50
174535/174535 [==============================] - 3s 16us/step - loss: 0.4961 - mean_squared_logarithmic_error: 0.0444
Epoch 3/50
174535/174535 [==============================] - 3s 16us/step - loss: 0.4796 - mean_squared_logarithmic_error: 0.0430
Epoch 4/50
174535/174535 [==============================] - 3s 16us/step - loss: 0.4733 - mean_squared_logarithmic_error: 0.0425
Epoch 5/50
174535/174535 [==============================] - 3s 16us/step - loss: 0.4701 - mean_squared_logarithmic_error: 0.0422
Epoch 6/50
174535/174535 [==============================] - 3s 16us/step - loss: 0.4673 - mean_squared_logarithmic_error: 0.0420
Epoch 7/50
174535/174535 [==============================] - 3s 16us/step - loss: 0.4648 - mean_squared_logarithmic_error: 0.0418
Epoch 8/50
174535/174535 [=======================

In [7]:
pred = model.evaluate(x = train_X, y = train_Y)
print('train RMSLE : ' + str(pred[1] ** 0.5))
pred = model.evaluate(x = test_X, y = test_Y)
print('test RMSLE : ' + str(pred[1] ** 0.5))

174535/174535 [==============================] - 13s 77us/step
train RMSLE : 0.20150257430787544
77573/77573 [==============================] - 6s 79us/step
test RMSLE : 0.20777332834349885
